In [28]:
!!auto-py-to-exe

^C


[]

In [23]:
import pandas as pd
gpa = pd.read_csv('sfrbx_gps_alm.csv', index_col=0)
gpe = pd.read_csv('sfrbx_gps_eph.csv', index_col=0)
gla = pd.read_csv('sfrbx_glonass_alm.csv', index_col=0)
gle = pd.read_csv('sfrbx_glonass_eph.csv', index_col=0)
with open('sfrbx_gps_data.txt', 'r') as file:
    gp_data = eval(file.read())

In [24]:
import pickle
from datetime import datetime
import Constants
t1 = datetime.now(tz=Constants.tz_utc)

t2 = datetime.now(tz=Constants.tz_utc)
print(t2-t1)  

0:00:00.001995


In [20]:
import numpy as np
from math import pi, floor, sin, cos, atan, sqrt, tan
import Constants

SV = 4
alm = gla.loc[SV-1]
eph = gle.loc[SV-1]
# print(alm)
print(f'SV: {SV} glonass')

def get_glo_year(N4, N):
    if 1 <= N <= 366:
        J=1
    elif 367 <= N <= 731:
        J = 2
    elif 732 <= N <= 1096:
        J = 3
    elif 1097<= N <= 1461:
        J = 4
    else:
        J = 0 # TODO - error
    return 1996 + 4 * (N4-1) + (J-1)

def get_glo_dNA(N4, NA, N):
    k = 1461 if N4 != 27 else 1460
    return N - NA - round((N-NA)/k)*k

#TODO: переименовать N в NA в преобраовании ГЛОНАСС 
# t1 = 81300 # 22:35
t1 = 23 * 3600 + 43 * 60
print(f'{t1//3600}h : {(t1%3600//60)}m')
N = 204 # 22.07.2024
# t1, N - время и день, на которые проводится расчет

def calc_glo_alm(alm, eph, t1, N):
    GM = Constants.mu
    ae = Constants.ae_glonass
    J20 = Constants.J20_glonass
    wz = Constants.OmegaEarthDot
    
    NA = eph.N
    N4 = eph.N4
    t_lambda = alm.t_lambda_n
    dT = alm.delta_T_n
    di = alm.delta_i_n
    dT_dot = alm.delta_T_dot_n
    e = alm.eps_n
    w_A = alm.omega_n
    lambda_A = alm.lambda_n
    
    # для проверки
    # N = 1453
    # t1 = 51300
    # NA = 1452
    # t_lambda = 33571.625
    # dT = 0.01953124999975
    # dT_dot= 6.103515625E-05
    # lambda_A = -0.293967247009277 * pi
    # w_A = 0.57867431640625 * pi
    # e = 0.000432968139648438
    # di = -0.00012947082519531 * pi
    
    # 1 - интервал прогноза dt_pr в сек
    dNa = get_glo_dNA(N4=N4, NA=NA, N=N)
    dt_pr = dNa * 86400 + (t1 - t_lambda)
    # print(1, dt_pr)
    
    # 2 - количество целых витков W на интервале прогноза
    Tdr_avg = 43200
    W = floor(dt_pr / (Tdr_avg + dT))
    # print(2, W)
    
    # 3 - текущее наклонение
    i_avg = 63 / 180 * pi
    i = di + i_avg
    # print(3, i)
    #TODO: результат не сходится с примеров в документации
    # #i = 1.1305666106990377
    
    # 4 - средний драконический период на витке W+1 и среднее движение
    Tdr = Tdr_avg + dT + (2*W + 1) * dT_dot
    n = 2*pi / Tdr
    # print(4, Tdr, n)
    
    # 5 - рассчитывается большая полуось последовательными приближениями (m=0, 1, 2...)
    T1 = Tdr
    a0 = -1
    a1 = 0
    p1 = 0
    while abs(a0-a1) > 1e-3:
        a1 = ((T1/2/pi)**2 * GM) ** (1.0/3.0)
        p1 = a1 * (1-e**2)
        Ttt = (2-2.5*sin(i)**2) * ((1-e**2) ** 1.5 / (1 + e * cos(w_A * pi)) ** 2) + (1 + e * cos(w_A * pi)) ** 3 / (1 - e ** 2)
        Tdown = 1 - 1.5 * J20 * (ae/p1)** 2 * Ttt
        T1 = Tdr / Tdown
        a0 = a1
    # T1, a1, p1
    # print(5, a1, p1, T1)
    
    # 6 - текущее значение долготы восходящего узла и аргумента перигея
    lmb = lambda_A - (wz + 1.5*J20*n*(ae/p1)**2*cos(i))* dt_pr
    w = w_A - 0.75 * J20 * n * (ae/p1)**2 * (1-5*cos(i)**2) * dt_pr
    # print(6, lmb, w)
    
    # 7 - среднее значение долготы на момент прохождения восходящего узла
    E0 = -2 * atan(sqrt((1-e)/(1+e)) * tan(w/2))
    L1 = w + E0 - e*sin(E0)
    # print(7, E0, L1)
    
    # 8 - текущее значение долготы НКА
    L = L1 + n * (dt_pr - (Tdr_avg + dT)*W - dT_dot*W**2)
    L2 = L
    # print(8, L)

    # 9 - для повышения точности, можно опустить
    # TODO: добавить п.9
    
    # 10 - эксцентрическая аномалия
    E1 = 10 + E0
    while abs(E1-E0) > 1e-9:
        E0 = E1
        E1 = L - w + e * sin(E0)
    E = E1
    # 11 истинняа аномалия и аргумент широты
    nu = 2 * atan( sqrt((1+e)/(1-e)) * tan(E/2))
    u = nu + w
    
    # 12 - координаты ЦМ в прямоугольной ск
    p = a1 * (1-e**2)
    r = p / (1+e * cos(nu))
    X = r * (cos(lmb)*cos(u)-sin(lmb)*sin(u)*cos(i))
    Y = r * (sin(lmb)*cos(u)+cos(lmb)*sin(u)*cos(i))
    Z = r * sin(u) * sin(i)
    
    # print(X, Y, Z)
    import Transformations
    # print(*Transformations.ecef2lla(X, Y, Z))
    return X, Y, Z

prMes = {
    7: 17931698.162640072,
    16: 17247315.440587413,
    17: 19352936.817768987,
    24: 21353624.2500098,
}
import Transformations
data = []
eph = gle.loc[6]
for svId in prMes.keys():
    alm = gla.loc[svId-1]
    # eph = gle.loc[svId-1]
    assert svId == alm.svId# and svId == eph.svId
    x, y, z = calc_glo_alm(alm, eph, t1, N)
    print(svId, tuple(np.array(Transformations.ecef2lla(x, y, z)).round(2)), round(prMes[svId], -3))
    data.append((x, y, z, prMes[svId]))


SV: 4 glonass
23h : 43m
7 (50.36, -9.68, 19157945.67) 17932000.0
16 (63.9, 69.69, 19158976.56) 17247000.0
17 (54.96, 125.55, 19158666.16) 19353000.0
24 (58.32, -144.88, 19152572.72) 21354000.0


In [9]:
import Minimizing
res1 = Minimizing.solve_navigation_task_SLSQP(data)
print(res1)
dt = res1.x[-1] / Constants.c
print(f'dt: {dt}')
import Transformations
lla = Transformations.ecef2lla(*res1.x[:-1])
print(lla)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 48929587592.090385
            Iterations: 86
            Function evaluations: 585
            Gradient evaluations: 86
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 48929587592.090385
       x: [ 2.828e+06  2.373e+06  3.724e+06 -3.456e+06]
     nit: 86
     jac: [-2.048e+04 -8.858e+04 -8.704e+04 -5.120e+02]
    nfev: 585
    njev: 86
dt: -0.011528917924381701
(45.479663799024976, 40.00324263089771, -1123275.200322042)


In [12]:
res2 = Minimizing.solve_navigation_task_LevMar(data)

print(res2)
dt = res2.x[-1] / Constants.c

print(f'dt: {dt}')
lla = Transformations.ecef2lla(*res2.x[:-1])
print(lla)
print(sum(res2.fun))

     message: `xtol` termination condition is satisfied.
     success: True
      status: 3
         fun: [ 7.471e-04  1.012e-03  3.840e-04  1.216e-04]
           x: [ 2.775e+06  2.175e+06  5.571e+06 -2.038e+06]
        cost: 8.721114388533032e-07
         jac: [[-3.639e-02  1.346e-02 -3.854e-02  5.467e-02]
               [-3.720e-03 -2.761e-02 -5.720e-02  6.362e-02]
               [ 2.072e-02 -1.789e-02 -2.808e-02  3.919e-02]
               [-1.296e-02 -9.325e-03  1.519e-02 -2.205e-02]]
        grad: [-2.457e-05 -2.589e-05 -9.561e-05  1.176e-04]
  optimality: 0.00011758257695835634
 active_mask: [0 0 0 0]
        nfev: 211
        njev: None
dt: -0.006799469126686438
(57.83694369172598, 38.08445483012054, 230385.68521757185)
0.0022645259861857475


In [11]:
res3 = Minimizing.solve_navigation_task_GO(data)
print(res3)
dt = res3.x[-1] / Constants.c

print(f'dt: {dt}')
lla = Transformations.ecef2lla(*res3.x[:-1])
print(lla)

 message: Optimization terminated successfully.
 success: True
     fun: 1.3877787807814457e-17
       x: [ 2.775e+06  2.175e+06  5.571e+06 -2.038e+06]
     nit: 295
    nfev: 17765
dt: -0.006799469217355879
(57.83694333146158, 38.08445499698492, 230385.69636427402)
